# **知识图谱构建**
大致分为以下几步

1、数据预处理

2、切分文段

3、输入大模型

4、反馈整合输出

5、分步重构

## **数据预处理**

目标：在不使用ocr的情况下，将文件转成合适长度的文本段落

数据源：规整的docx类型，以及内容为图像的pdf文件

开源库：**python-docx**，内容按段落切分，没有“页”以及“目录”的操作api

**PyMuPDF**，内容按页切分，输入文件需为含有文本信息的pdf文件

处理方式：将docx转成pdf然后按照pdf进行数据读取和按页切分

In [9]:
# 将目录数据转成实体关系数据以及切片数据
from buildKG.str2json import indexstr2json,str2json
from buildKG.docx2request import group_paragraphs_by_token_limit,extract_text_from_pages,group_page_by_token_limit
from buildKG.forest import Node,Edge,Forest

from http import HTTPStatus
from dashscope import Generation
from dashscope.api_entities.dashscope_response import Role
import dashscope
import time
import random
import os
import fitz  # PyMuPDF
import copy
dashscope.api_key = "sk-8cdfdc6958a7408492d465269349299f"

下面不好用，要windows系统，我直接wps转pdf,当然不转直接拿langchain的Unstructured来做也可以，同样问题也有不如WPS docx转pdf 人工选取 目录页数快

In [2]:
# # docx转pdf 
# from docx2pdf import convert

# # 指定.docx文件路径和.pdf输出路径
# docx_file_path = './data/（139） 3D打印技术基础（第三版）-朱红,易杰,谢丹-华中科技大学出版社.docx'
# pdf_output_path = './data_pdf/（139） 3D打印技术基础（第三版）-朱红,易杰,谢丹-华中科技大学出版社.pdf'

# # 执行转换
# convert(docx_file_path, pdf_output_path)

## **文段切分**
目标：选取合适大小的文本段落为单元任务，去让大模型进行归纳总结

1、直接按长度进行划分：很碎，造成野节点非常多，不能作为一个方案

2、按章节进行划分：不够细致，大多数只挺留在了第一级

3、按小节进行划分：很深，很杂，会把很多一个定义下的大段文本作为输出，同样野节点非常多

处理方式：

按章节进行划分的方式，以章节名作为一级子节点，后续野子节点置于一级子节点上

In [10]:
# 目录起始页
start_page = 12
end_page = 15
pdf_output_path = './data/（139） 3D打印技术基础（第三版）-朱红,易杰,谢丹-华中科技大学出版社.pdf'

**将切分好的目录数据给到大模型，让大模型总结目录**

In [4]:
# 执行目录提取
request=extract_text_from_pages(pdf_output_path, start_page, end_page)

nodes_array=[]
edges_array=[]
pages_array=[]
log=[]
prompt = """
    在接下来的对话中，你需要遵循以下规则：
    你是一台用于处理文本数据的AI系统。你的任务是处理我上传的文本文件，并按照以下要求进行操作：
    - 输出标准JSON格式的切片结果。
    - 剔除文件中的所有乱码（乱码不包括英文）。
    - 处理英文的时候 请保留英文的原格式，包括括号。
    - 根据你的判断，可以筛选无意义的、与目录无关的句子。但是要确保没有改变原意，保留了原本的信息量。
    - 确保没有信息遗漏，不要缺省任何有效信息。
    - 按照章节、小节，对内容进行切片，构建树状JSON结构，请确保你输出的JSON的格式规范。
    - 请严格参考格式示例:
    {
      "slices": [
        {
      "chapter_name": "(章节去除编号对应名称）",
      "page": "（章节对应页码）",
      "entries": [
        {
          "entry_name": "（小节去除编号对应名称）",
          "page": "（小节对应页码）",
        }
        {
          "entry_name": "（小节去除编号对应名称）",
          "page": "（小节对应页码）",
        }
      ]
    },
      ]
    }
    - 确保每一个条目都被完整地处理，不漏切片。
    - 对chapter_name、entries中的属性，请不要带上任何标号性质的前缀，并对你提取到的名称进行概括
    - 不要输出subentries
    - 你不需要确认收到，请从第一条讯息开始，直接开始生成完整准确的输出，不允许输出json之外的内容
"""

messages = [{'role': Role.SYSTEM, 'content': prompt},
            {'role': Role.USER, 'content': request}]
response = Generation.call(
    Generation.Models.qwen_max_longcontext,
    messages=messages,
    result_format='message',  # set the result to be "message" format.
)
if response.status_code == HTTPStatus.OK:
    # append result to messages.
    raw_string=response.output.choices[0]['message']['content']
    print(raw_string)
    nodes_array,edges_array,pages_array,pages_part_array=indexstr2json(raw_string)
else:
    print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
        response.request_id, response.status_code,
        response.code, response.message
    ))

{
  "slices": [
    {
      "chapter_name": "绪论",
      "page": "1",
      "entries": [
        {
          "entry_name": "3D打印概述",
          "page": "1"
        },
        {
          "entry_name": "3D打印工艺分类",
          "page": "2"
        },
        {
          "entry_name": "3D打印技术优势",
          "page": "4"
        },
        {
          "entry_name": "3D打印技术的意义",
          "page": "5"
        },
        {
          "entry_name": "3D打印发展趋势",
          "page": "6"
        },
        {
          "entry_name": "3D打印的未来",
          "page": "8"
        }
      ]
    },
    {
      "chapter_name": "3D打印基础理论",
      "page": "10",
      "entries": [
        {
          "entry_name": "3D打印数据",
          "page": "10"
        },
        {
          "entry_name": "模型处理",
          "page": "30"
        }
      ]
    },
    {
      "chapter_name": "光固化成形",
      "page": "43",
      "entries": [
        {
          "entry_name": "SLA成形原理及特点",
          "page": "43"
        },
        {
          "

**章节列表**

In [5]:
# 检查LLM输出是否带有章节号，是否带有小结等会污染节点的内容
part_array=copy.copy(nodes_array)
print(len(part_array))

47


In [6]:
nodes_array

['绪论',
 '3D打印概述',
 '3D打印工艺分类',
 '3D打印技术优势',
 '3D打印技术的意义',
 '3D打印发展趋势',
 '3D打印的未来',
 '3D打印基础理论',
 '3D打印数据',
 '模型处理',
 '光固化成形',
 'SLA成形原理及特点',
 'SLA成形材料',
 'SLA成形设备',
 'SLA成形件后处理',
 '熔融沉积制造',
 'FDM成形原理及特点',
 'FDM成形材料',
 'FDM成形设备',
 'FDM成形件后处理',
 '激光选区烧结',
 'SLS成形原理及特点',
 'SLS成形材料',
 'SLS成形设备',
 'SLS成形件后处理',
 '激光选区熔化',
 'SLM成形原理及特点',
 'SLM成形材料',
 'SLM成形设备',
 'SLM成形件后处理',
 '三维打印',
 '3DP成形原理及特点',
 '3DP成形材料',
 '3DP成形设备',
 '3DP成形件后处理',
 '其他成形技术',
 '分层实体制造',
 '形状沉积制造',
 '数字化光照加工',
 '3D打印技术的应用',
 '工业制造领域中的应用',
 '文化艺术领域中的应用',
 '生物医学领域中的应用',
 '建筑领域中的应用',
 '食品领域中的应用',
 '教育领域中的应用',
 '各主流3D打印成形技术的应用']

**对应页数**

In [7]:
# 这段代码不能重复跑，pages_array需要重新生成
# pages_array=[page+end_page for page in pages_array]
# pages_array=sorted(pages_array)
pages_array=[16, 17, 19, 20, 21, 23, 25, 45, 58, 60, 65, 77, 80, 82, 90, 100, 101, 102, 107, 119, 123, 125, 129, 138, 140, 141, 144, 151, 153, 155, 157, 161, 171, 179, 183, 185, 186, 188]
# 补充尾页信息
# 打开 PDF 文件
pdf_document = fitz.open(pdf_output_path)
    # 获取总页数
total_pages = pdf_document.page_count
pages_array.append(total_pages)

## **3.输入提示词设计**

目标：指引大模型更好的利用资源优势实现按照固定格式的输出

携带上下文：影响输出格式，输出很杂，会导致实体中有大段文本出现

不携带上下文：不够细致

改进方向：
围绕目录信息，将章节内部信息给到大模型，让大模型去执行抽取任务

In [8]:
node_array=copy.copy(part_array)
debug_array=[]
part_name=""
prompt = f"""
    在接下来的对话中，你需要遵循以下规则：
    你是一个擅长读书并总结的人，你的任务是处理我上传的课本选段，并按照以下要求进行操作：
    - 输出标准JSON格式字符串。
    - 剔除文件中的所有乱码（乱码不包括英文）。
    - 处理英文的时候 请保留英文的原格式，包括括号。
    - 我会告诉你这段文字的主题，你可以围绕着这个主题开始一层一层地梳理知识结构
    - 你需要对文本内知识点进行提取，并列成列表，形成json字符串的point字段。
    - 当你提取到一个知识点，在上下文寻找与他相关联的知识点，并进行提取
    - 你需要对知识点与知识点之间的属于关系进行提取，将有属于关系的知识点两两之间形成(知识点1,属于，知识点2)的元组，并以列表的形式存进json字符串的edge字段
    - 像"一"、"二"或是"第一"、"第二"等形式的序号字符，可用于帮助定位实体。标题之间的层级结构，也可以帮助构建属于关系。
    - 这一节的名字叫{part_name},请围绕{part_name}进行知识点提取和属于关系识别
    - 格式参考示例:
    {{
        "point": ["数学","线性代数","矩阵"],
        "edge":[["线性代数","属于","数学"],["矩阵","属于","线性代数"]]
    }}
    - 请保证仅提取属于关系，被提取出的知识点一定有相应的关系。
    你不需要确认收到，或是做出过多解释，直接开始生成完整准确的输出。
"""
    # 下面这段原本是写在提示词里面的，表征上下文，但是由于我们将提取目录中，构建关系的部分删去了，这部分写入提示词会导致边生成不出来，故注释掉
    # - 同时你可以参考上下文中已经提取好的实例和关系,在生成答案的时候，你必须保证这些实例和关系不能改变，只能在这些基础之上新增实体和关系：
    # {{
    #     "point": {nodes_array},
    #     "edge":  {edges_array}
    # }}
    # 实测加了这段话生成的内容边多了，但是很杂，影响最后生成结果且会造成各种生成的答案格式不正确等问题；但是理论上来说，我们需要上下文提示，下一期工程再改吧
for i in range(0,len(pages_array)-1):
    print("————————————第"+str(i+1)+"章————————————")
    start=pages_array[i]
    end=pages_array[i+1]
    part_name=pages_part_array[i]
    prompt = f"""
    在接下来的对话中，你需要遵循以下规则：
    你是一个擅长读书并总结的人，你的任务是处理我上传的课本选段，并按照以下要求进行操作：
    - 输出标准JSON格式字符串。
    - 剔除文件中的所有乱码（乱码不包括英文）。
    - 处理英文的时候 请保留英文的原格式，包括括号。
    - 我会告诉你这段文字的主题，你可以围绕着这个主题开始一层一层地梳理知识结构
    - 你需要对文本内知识点进行提取，并列成列表，形成json字符串的point字段。
    - 当你提取到一个知识点，在上下文寻找与他相关联的知识点，并进行提取
    - 你需要对知识点与知识点之间的属于关系进行提取，将有属于关系的知识点两两之间形成(知识点1,属于，知识点2)的元组，并以列表的形式存进json字符串的edge字段
    - 像"一"、"二"或是"第一"、"第二"等形式的序号字符，可用于帮助定位实体。标题之间的层级结构，也可以帮助构建属于关系。
    - 这一节的名字叫{part_name},请围绕{part_name}进行知识点提取和属于关系识别
    - 格式参考示例:
    {{
        "point": ["数学","线性代数","矩阵"],
        "edge":[["线性代数","属于","数学"],["矩阵","属于","线性代数"]]
    }}
    - 请保证仅提取属于关系，被提取出的知识点一定有相应的关系。
    你不需要确认收到，或是做出过多解释，直接开始生成完整准确的输出。
    """
    # 50000是token数量，可以用来选择单元切片的大小
    # 经过我的测试，小节所表现出的效果不如章节，很乱很杂，当然也有可能是因为上下文提示词的原因
    contents=group_page_by_token_limit(pdf_output_path,50000, start, end)
    for content_id,content in enumerate(contents):
        request=""
        for paragraph_id,paragraph in enumerate(content):
            request+=paragraph
            request+='\n'
        # messages.append({'role': Role.USER, 'content': request})
        messages = [{'role': Role.SYSTEM, 'content': prompt},
                {'role': Role.USER, 'content': request}]
        response = Generation.call(
            Generation.Models.qwen_max_longcontext,
            messages=messages,
            result_format='message',  # set the result to be "message" format.
        )
        if response.status_code == HTTPStatus.OK:
            raw_string=response.output.choices[0]['message']['content']
            print(raw_string)
            print(response.usage["total_tokens"])
            # append result to messages.
            nodes_array_n,edges_array_n=str2json(raw_string)
            nodes_array+=[node for node in nodes_array_n if node not in nodes_array]
            edges_array+=[edge for edge in edges_array_n if edge not in edges_array]
            log.append((i,nodes_array_n,edges_array_n))
            if nodes_array_n ==[] or edges_array_n==[]:
                debug_array.append(i)
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))
            debug_array.append(i)
            break
        # api限制是1分钟调用一次
        secs = random.normalvariate(60, 0.4)
        if secs <= 40 or secs >= 70:
            secs =50   # 太小或太大则重置为平均值
        print("sleep:"+str(secs)+"秒")
        time.sleep(secs)       

————————————第1章————————————


KeyboardInterrupt: 

## **4.反馈整合，xmind输出**

构建了森林、边、节点类，用节点表征实体，用边表征属于关系，relationship表征其他关系
遇到问题：
1、有的边添加时间过长，但是从边上看暂未发现原因
2、只添加其他关系不添加从属关系不能使节点被显示，我们在其他关系上也加了一个从属关系这会导致从属混乱，程序执行出错

In [9]:
# 构建xmind
forest = Forest()
forest.build_forest(part_array,[],None)
for l in log:
    id,nodes_array_i,edges_array_i=l
    part=part_array[id]
    print(part)
    forest.build_forest(nodes_array_i,edges_array_i,part)
forest.build_kg("3D打印技术基础")

None
绪论
['3D打印', '属于', '增材制造']
['增材制造', '又称为', 'additive manufacturing']
['additive manufacturing', '缩写', 'AM技术']
['AM技术', '体现', '信息技术']
['AM技术', '体现', '先进材料技术']
['AM技术', '体现', '数字制造技术']
['AM技术', '属于', '交叉融合']
['减材制造', '对比', '3D打印']
['材料去除', '例如', '切削加工']
['自下而上', '描述', '材料累加制造方法']
['材料累加制造', '等于', 'material increase manufacturing']
['快速原型', '等于', 'rapid prototyping']
['快速原型', '缩写', 'RP']
['分层制造', '等于', 'layered manufacturing']
['分层制造', '缩写', 'LM']
['实体自由成形制造', '等于', 'solid freeform fabrication']
['实体自由成形制造', '缩写', 'SFF']
['3D打印', '定义由', '美国材料与试验协会']
['3D打印', '定义由', 'ASTM']
['3D打印', '定义由', 'F42国际委员会']
['3D打印', '定义由', '3D打印技术委员会']
['窄义层次上的3D打印', '使用', '打印头']
['窄义层次上的3D打印', '使用', '喷嘴']
['窄义层次上的3D打印', '使用', '打印技术设备']
['窄义层次上的3D打印', '等于', '相对价格或总体功能低端的增材制造设备']
['广义层次上的3D打印', '等于', '增材制造技术']
['自由制造', '实现方式', '3D打印技术']
['复杂结构零件', '解决方法', '3D打印技术']
['加工工序', '减少方式', '3D打印技术']
['加工周期', '缩短方式', '3D打印技术']
['结构复杂产品', '优势体现', '3D打印技术优势']
['3D打印', '应用领域', '模具']
['3D打印', '应用领域', '汽车']
['3D打印', '应用领域'

NotFoundErr: 

## **5.分步重构**
主要是两方面的工作：
1.选段重新输入大模型
2.大模型生成结果保存

**大模型生成结果保存**

In [19]:
import pickle
with open('node.pkl', 'wb') as f:
    pickle.dump(nodes_array, f)
with open('edge.pkl', 'wb') as f:
    pickle.dump(edges_array, f)
with open('log.pkl', 'wb') as f:
    pickle.dump(log, f)
with open('part_array.pkl', 'wb') as f:
    pickle.dump(part_array, f)

In [4]:
import pickle
with open('node.pkl', 'rb') as f:
    nodes_array = pickle.load(f)
with open('edge.pkl', 'rb') as f:
    edges_array = pickle.load(f)
with open('log.pkl', 'rb') as f:
    log = pickle.load(f)
with open('part_array.pkl', 'rb') as f:
    part_array = pickle.load(f)

**选段重新输入大模型**

In [2]:
# 这块主要是为了处理出现错误的流程,当然如果觉得生成效果不够理想，也可以通过更改debug_array去将选中章节添加入重新生成的段落
# 错误出现的主要原因是：1、文段过长，2、输出格式出错，3、请求过快或网络连接不稳定，4、请求文段中含有敏感文段
# debug_array=[4, 9, 11, 13, 17, 25]
debug_array=list(set(debug_array))

In [8]:
debug_id=0
while debug_id<len(debug_array):
    i=debug_array[debug_id]
    start=pages_array[i]
    end=pages_array[i+1]
    contents=group_page_by_token_limit(pdf_output_path,50000, start, end)
    debug_state=True
    for content_id,content in enumerate(contents):
        request=""
        print("————————————第"+str(content_id)+"次请求————————————")
        for paragraph_id,paragraph in enumerate(content):
            request+=paragraph
            request+='\n'
        # messages.append({'role': Role.USER, 'content': request})
        messages = [{'role': Role.SYSTEM, 'content': prompt},
                {'role': Role.USER, 'content': request}]
        response = Generation.call(
            Generation.Models.qwen_max_longcontext,
            messages=messages,
            result_format='message',  # set the result to be "message" format.
        )
        if response.status_code == HTTPStatus.OK:
            raw_string=response.output.choices[0]['message']['content']
            print(raw_string)
            print(response.usage["total_tokens"])
            # append result to messages.
            nodes_array_n,edges_array_n=str2json(raw_string.replace("'", '"'))
            nodes_array+=[node for node in nodes_array_n if node not in nodes_array]
            edges_array+=[edge for edge in edges_array_n if edge not in edges_array]
            log.append((i,nodes_array_n,edges_array_n))
        else:
            print('Request id: %s, Status code: %s, error code: %s, error message: %s' % (
                response.request_id, response.status_code,
                response.code, response.message
            ))
            debug_state=False
            break
        secs = random.normalvariate(80, 0.4)
        if secs <= 60 or secs >= 120:
            secs =80   # 太小则重置为平均值
        print("sleep:"+str(secs)+"秒")
        time.sleep(secs)
    if debug_state:
        debug_id+=1

————————————第0次请求————————————


NameError: name 'prompt' is not defined